In [1]:
%pylab notebook
from spi_helper import *
sys.path.append("../..")
from common import *

Populating the interactive namespace from numpy and matplotlib


# Connec to Zedboard PL

In [2]:
r = conLitexServer('../build/csr.csv')

Connected to Port 1234
HMC7044 spi bvridge 2019-09-09 18:01:17


# Setup AD9174

In [3]:
ad = AdSpi(r)
ad.wr(0x000, 0x81)  # Soft reset
ad.wr(0x000, 0x3C)  # 4 - wire SPI mode + ADDRINC
ad.wr(0x705, 0x01)  # Enable boot loader
ad.wr(0x090, 0x00)  # Power on DACs and bias supply
# Print product ID
print(hex(ad.rr(0x004)))
print(hex(ad.rr(0x005)))

0x74
0x91


# Setup HMC7044
  * External VCO input
  * clkout12 = f_vco / 10 = 128 MHz

In [5]:
hmc = HmcSpi(r)

hmc.wr(0x000, 1)  # reset
hmc.wr(0x000, 0)

hmc.wr(0x054, 0)  # Disable SDATA driver (uni-direct. buffer)
hmc.wr(0x001, (1 << 6) | (1 << 5)) # High performance dividers / PLL

# VCO Selection
# 0 Internal disabled/external
# 1 High
# 2 Low
VCO_SELECT = 3
hmc.wr(0x003, (0 << VCO_SELECT))

# Enable output channel 12 and 13
hmc.wr(0x004, (1 << (12 // 2)))

# clkin1 as external VCO input
hmc.wr(0x005, (1 << 5))

# Magic numbers from datasheet Table 74
hmc.wr(0x09F, 0x4D)
hmc.wr(0x0A0, 0xDF)
hmc.wr(0x0A5, 0x06)
hmc.wr(0x0A8, 0x06)
hmc.wr(0x0B0, 0x04)

hmc.setupChannel(12, 10)

In [6]:
r.close()